%pip install selenium python-dotenv

In [1]:
import os
import csv
import time
import random
import logging
from dotenv import load_dotenv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)
load_dotenv()

True

In [ ]:
LINKEDIN_EMAIL = os.getenv("LINKEDIN_EMAIL")
LINKEDIN_PASSWORD = os.getenv("LINKEDIN_PASSWORD")
PROFILE_URL = [
    "https://www.linkedin.com/in/akshaya-ravichandran-24671b321/",
    "https://www.linkedin.com/in/subendran-s-9a3192253/",
    "https://www.linkedin.com/in/vignesh-kumar-sp-1a412827b/",
    "https://www.linkedin.com/in/vigneswari-d-28809027b/",
    "https://www.linkedin.com/in/guru-prasath-m130105/",
    "https://www.linkedin.com/in/sowmiya-thejanathan-32b0442b4/",
    "https://www.linkedin.com/in/claribel-hermia/",
    "https://www.linkedin.com/in/sivapriyal-karikalan-21a17332b/",
    "https://www.linkedin.com/in/madhumitha-csv/",
    "https://www.linkedin.com/in/dhivia-bharathi-50512827b/",
    "https://www.linkedin.com/in/sivasri-v-06b49727b/",
    "https://www.linkedin.com/in/swetha-r-22029126a/",
    "https://www.linkedin.com/in/yuva-haran-18109827b/",
    "https://www.linkedin.com/in/maadhavprasad75/",
    "https://www.linkedin.com/in/vignesh19032005/",
    "https://www.linkedin.com/in/kamalesh-b-14a1b32a4/",
    "https://www.linkedin.com/in/visshal-prasath-m-626b97304/",
    "https://www.linkedin.com/in/m-sabarish-53b703221/",
    "https://www.linkedin.com/in/vasanth-v-91a13427b/",
    "https://www.linkedin.com/in/hemanth-yarra/"
    ]

In [ ]:
class LinkedInScraper:
    def __init__(self):
        self.setup_driver()
        self.wait = WebDriverWait(self.driver, 20)
        
    def setup_driver(self):
        options = webdriver.ChromeOptions()
        options.add_argument('--disable-blink-features=AutomationControlled')
        options.add_argument('--start-maximized')
        options.add_experimental_option("excludeSwitches", ["enable-automation"])
        options.add_experimental_option('useAutomationExtension', False)
        self.driver = webdriver.Chrome(options=options)
        
    def human_delay(self, min_sec=1, max_sec=3):
        time.sleep(random.uniform(min_sec, max_sec))
        
    def smooth_scroll(self, scrolls=5):
        for i in range(scrolls):
            scroll_height = 800 + random.randint(-100, 100)
            self.driver.execute_script(f"window.scrollBy(0, {scroll_height});")
            self.human_delay(0.5, 1.5)
        self.driver.execute_script("window.scrollTo(0, 0);")
        self.human_delay(1, 2)
    
    def login(self, email, password):
        try:
            logger.info("Logging into LinkedIn...")
            self.driver.get("https://www.linkedin.com/login")
            self.wait.until(EC.presence_of_element_located((By.ID, "username")))
            username_field = self.driver.find_element(By.ID, "username")
            for char in email:
                username_field.send_keys(char)
                time.sleep(random.uniform(0.05, 0.15))
            self.human_delay(0.5, 1)
            password_field = self.driver.find_element(By.ID, "password")
            for char in password:
                password_field.send_keys(char)
                time.sleep(random.uniform(0.05, 0.15))
            self.human_delay(0.5, 1)
            self.driver.find_element(By.XPATH, "//button[@type='submit']").click()
            self.human_delay(3, 5)
            logger.info("Successfully logged in!")
            
        except Exception as e:
            logger.error(f"Login failed: {str(e)}")
            raise
    
    def safe_find_element(self, by, selector, multiple=False):
        try:
            if multiple:
                elements = self.driver.find_elements(by, selector)
                return [el.text.strip() for el in elements if el.text.strip()]
            else:
                element = self.driver.find_element(by, selector)
                return element.text.strip()
        except (NoSuchElementException, TimeoutException):
            return [] if multiple else ""
    
    def extract_basic_info(self):
        data = {}
        name_selectors = [
            (By.CSS_SELECTOR, "h1.text-heading-xlarge"),
            (By.XPATH, "//h1[contains(@class, 'text-heading-xlarge')]"),
            (By.CSS_SELECTOR, "div.mt2.relative h1")
        ]
        for by, selector in name_selectors:
            data['name'] = self.safe_find_element(by, selector)
            if data['name']:
                break
        
        # Headline
        headline_selectors = [
            (By.CSS_SELECTOR, "div.text-body-medium.break-words"),
            (By.XPATH, "//div[contains(@class, 'text-body-medium')]"),
        ]
        
        for by, selector in headline_selectors:
            data['headline'] = self.safe_find_element(by, selector)
            if data['headline']:
                break
        location_selectors = [
            (By.CSS_SELECTOR, "span.text-body-small.inline.t-black--light.break-words"),
            (By.XPATH, "//span[contains(@class, 'text-body-small') and contains(@class, 't-black--light')]"),
        ]
        
        for by, selector in location_selectors:
            data['location'] = self.safe_find_element(by, selector)
            if data['location']:
                break
        
        return data
    
    def extract_about(self):
        try:
            show_more_buttons = self.driver.find_elements(
                By.XPATH, 
                "//button[contains(@aria-label, 'Show more') or contains(@aria-label, 'more')]"
            )
            if show_more_buttons:
                self.driver.execute_script("arguments[0].click();", show_more_buttons[0])
                self.human_delay(0.5, 1)
        except:
            pass
        about_selectors = [
            (By.CSS_SELECTOR, "div.display-flex.ph5.pv3 div.inline-show-more-text span[aria-hidden='true']"),
            (By.XPATH, "//section[contains(@class, 'artdeco-card')]//div[contains(@class, 'inline-show-more-text')]"),
        ]
        for by, selector in about_selectors:
            about = self.safe_find_element(by, selector)
            if about:
                return about
        
        return ""
    
    def extract_experience(self):
        experiences = []
        try:
            exp_items = self.driver.find_elements(
                By.XPATH,
                "//section[contains(@id, 'experience')]//ul/li[contains(@class, 'artdeco-list__item')]"
            )
            for item in exp_items[:5]:
                try:
                    title = self.safe_find_element(
                        By.CSS_SELECTOR,
                        "div.display-flex.flex-column.full-width span[aria-hidden='true']",
                    ) if item else ""
                    company = ""
                    company_elements = item.find_elements(
                        By.CSS_SELECTOR,
                        "span.t-14.t-normal span[aria-hidden='true']"
                    )
                    if company_elements and len(company_elements) > 0:
                        company = company_elements[0].text.strip()
                    duration = ""
                    duration_elements = item.find_elements(
                        By.CSS_SELECTOR,
                        "span.t-14.t-normal.t-black--light span[aria-hidden='true']"
                    )
                    if duration_elements and len(duration_elements) > 0:
                        duration = duration_elements[0].text.strip()
                    if title or company:
                        exp_text = f"{title} at {company}" if title and company else title or company
                        if duration:
                            exp_text += f" ({duration})"
                        experiences.append(exp_text)
                except Exception as e:
                    logger.debug(f"Error extracting experience item: {str(e)}")
                    continue
        except Exception as e:
            logger.debug(f"Error in experience section: {str(e)}")
        
        return experiences
    
    def extract_education(self):
        educations = []
        try:
            edu_items = self.driver.find_elements(
                By.XPATH,
                "//section[contains(@id, 'education')]//ul/li[contains(@class, 'artdeco-list__item')]"
            )
            for item in edu_items[:3]:
                try:
                    edu_parts = item.find_elements(
                        By.CSS_SELECTOR,
                        "span[aria-hidden='true']"
                    )
                    if edu_parts:
                        edu_text = " - ".join([part.text.strip() for part in edu_parts[:3] if part.text.strip()])
                        if edu_text:
                            educations.append(edu_text)
                except Exception as e:
                    logger.debug(f"Error extracting education item: {str(e)}")
                    continue
        except Exception as e:
            logger.debug(f"Error in education section: {str(e)}")
        
        return educations
    
    def extract_skills(self):
        skills = []
        try:
            show_all_buttons = self.driver.find_elements(
                By.XPATH,
                "//section[contains(@id, 'skills')]//a[contains(@href, 'skills')]"
            )
            if show_all_buttons:
                self.driver.execute_script("arguments[0].click();", show_all_buttons[0])
                self.human_delay(2, 3)
                skill_elements = self.driver.find_elements(
                    By.XPATH,
                    "//div[contains(@class, 'scaffold-finite-scroll')]//span[contains(@class, 't-bold')]"
                )
                skills = [el.text.strip() for el in skill_elements[:20] if el.text.strip()]
                try:
                    close_button = self.driver.find_element(
                        By.XPATH,
                        "//button[@aria-label='Dismiss']"
                    )
                    close_button.click()
                    self.human_delay(0.5, 1)
                except:
                    pass
            if not skills:
                skills = self.safe_find_element(
                    By.XPATH,
                    "//section[contains(@id, 'skills')]//span[contains(@class, 't-bold')]",
                    multiple=True
                )[:20]
                
        except Exception as e:
            logger.debug(f"Error extracting skills: {str(e)}")
        
        return skills
    
    def scrape_profile(self, profile_url):
        logger.info(f"Scraping profile: {profile_url}")
        
        try:
            self.driver.get(profile_url)
            self.wait.until(EC.presence_of_element_located((By.TAG_NAME, "main")))
            self.human_delay(2, 3)
            self.smooth_scroll(scrolls=8)
            data = {"profile_url": profile_url}
            basic_info = self.extract_basic_info()
            data.update(basic_info)
            data['about'] = self.extract_about()
            experiences = self.extract_experience()
            data['exp_title'] = experiences[0] if experiences else ""
            data['exp_company'] = experiences[1] if len(experiences) > 1 else ""
            data['experience_full'] = " | ".join(experiences)
            educations = self.extract_education()
            data['education'] = educations[0] if educations else ""
            data['education_full'] = " | ".join(educations)
            skills = self.extract_skills()
            data['skills'] = ", ".join(skills[:10]) if skills else ""
            data['skills_full'] = ", ".join(skills)

            logger.info(f"Successfully scraped: {data.get('name', 'Unknown')}")
            return data
            
        except Exception as e:
            logger.error(f"Error scraping {profile_url}: {str(e)}")
            return {"profile_url": profile_url, "error": str(e)}
    
    def save_to_csv(self, data, filename="linkedin_scraped_enhanced.csv"):
        if not data:
            return
        file_exists = os.path.isfile(filename)
        fieldnames = [
            'profile_url', 'name', 'headline', 'location', 'about',
            'exp_title', 'exp_company', 'experience_full',
            'education', 'education_full', 'skills', 'skills_full'
        ]
        
        with open(filename, 'a', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=fieldnames, extrasaction='ignore')
            if not file_exists:
                writer.writeheader()
            writer.writerow(data)
        logger.info(f"Data saved to {filename}")
    
    def run(self, profile_urls):
        try:
            self.login(LINKEDIN_EMAIL, LINKEDIN_PASSWORD)
            for i, url in enumerate(profile_urls, 1):
                logger.info(f"Processing profile {i}/{len(profile_urls)}")
                
                data = self.scrape_profile(url)
                self.save_to_csv(data)
                if i < len(profile_urls):
                    delay = random.uniform(5, 10)
                    logger.info(f"Waiting {delay:.1f} seconds before next profile...")
                    time.sleep(delay)
            
            logger.info("Scraping completed successfully!")
            
        except Exception as e:
            logger.error(f"Scraping failed: {str(e)}")
            raise
        finally:
            self.driver.quit()

In [4]:
if not LINKEDIN_EMAIL or not LINKEDIN_PASSWORD:
    logger.error("LinkedIn credentials not found in .env file")
    exit(1)
    
if not PROFILE_URL:
    logger.warning("No profile URLs provided. Please add URLs to PROFILE_URLS list.")
    exit(1)
    
scraper = LinkedInScraper()
scraper.run(PROFILE_URL)

2025-11-08 14:41:14,172 - INFO - Logging into LinkedIn...
2025-11-08 14:43:33,318 - INFO - Successfully logged in!
2025-11-08 14:43:33,319 - INFO - Processing profile 1/20
2025-11-08 14:43:33,320 - INFO - Scraping profile: https://www.linkedin.com/in/akshaya-ravichandran-24671b321/
2025-11-08 14:44:03,110 - INFO - Successfully scraped: Akshaya Ravichandran
2025-11-08 14:44:03,112 - INFO - Data saved to linkedin_scraped_enhanced.csv
2025-11-08 14:44:03,113 - INFO - Waiting 7.4 seconds before next profile...
2025-11-08 14:44:10,510 - INFO - Processing profile 2/20
2025-11-08 14:44:10,513 - INFO - Scraping profile: https://www.linkedin.com/in/subendran-s-9a3192253/
2025-11-08 14:44:25,734 - INFO - Successfully scraped: Subendran S
2025-11-08 14:44:25,737 - INFO - Data saved to linkedin_scraped_enhanced.csv
2025-11-08 14:44:25,738 - INFO - Waiting 9.7 seconds before next profile...
2025-11-08 14:44:35,465 - INFO - Processing profile 3/20
2025-11-08 14:44:35,466 - INFO - Scraping profile: h